In [1]:
# Library imports
import pandas as pd
import numpy as np
import json

In [2]:
# Turn off the SettingWithCopyWarning (do this post-dev only)
pd.set_option('mode.chained_assignment', None)

In [3]:
# Read the datasets
df1 = pd.read_csv('../data/primary_dataset.csv')
df2 = pd.read_csv('../data/secondary_dataset.csv')

## Primary dataset processing

In [4]:
df1.columns

Index(['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore'],
      dtype='object')

In [5]:
# Convert to a python list
df1['Actors List'] = df1.Actors.map(lambda x: [i.strip() for i in x.split(",")])
# Convert movie titles to lower case (for convenience)
df1['Title'] = df1['Title'].map(str.lower)

In [6]:
# Keep only relevant columns (for now - add more fields for extra filter features)
df1_relevant = df1[['Title', 'Actors List']]
# Standardize column names
df1_relevant.columns = ['title', 'actors']
df1_relevant.head()

,title,actors
0,guardians of the galaxy,"[Chris Pratt, Vin Diesel, Bradley Cooper, Zoe ..."
1,prometheus,"[Noomi Rapace, Logan Marshall-Green, Michael F..."
2,split,"[James McAvoy, Anya Taylor-Joy, Haley Lu Richa..."
3,sing,"[Matthew McConaughey, Reese Witherspoon, Seth ..."
4,suicide squad,"[Will Smith, Jared Leto, Margot Robbie, Viola ..."


## Secondary dataset processing

In [7]:
df2.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [8]:
# We have the "\xa0" character at the end ofthe titles: we want to get rid of it
df2.movie_title = df2.movie_title.replace(u'\xa0', u'', regex=True)
# Combine columns and convert to python list
df2['actors_list'] = df2[['actor_1_name', 'actor_2_name', 'actor_3_name']].apply(lambda x: ','.join(x.dropna()), axis=1)
df2['actors_list'] = df2['actors_list'].map(lambda x: [i.strip() for i in x.split(",")])
# Convert movie titles to lower case (for convenience)
df2['movie_title'] = df2['movie_title'].map(str.lower)
# Drop rows with NaN values
df2 = df2.dropna()

In [9]:
# Keep only relevant columns (for now - add more fields for extra filter features)
df2_relevant = df2[['movie_title', 'director_name', 'actors_list', 'imdb_score', 'gross', 'budget']]
# Standardize column names
df2_relevant.columns = ['title', 'director', 'actors', 'imdb_rating', 'revenue', 'budget']
df2_relevant.head()

,title,director,actors,imdb_rating,revenue,budget
0,avatar,James Cameron,"[CCH Pounder, Joel David Moore, Wes Studi]",7.9,760505847.0,237000000.0
1,pirates of the caribbean: at world's end,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Jack Davenport]",7.1,309404152.0,300000000.0
2,spectre,Sam Mendes,"[Christoph Waltz, Rory Kinnear, Stephanie Sigman]",6.8,200074175.0,245000000.0
3,the dark knight rises,Christopher Nolan,"[Tom Hardy, Christian Bale, Joseph Gordon-Levitt]",8.5,448130642.0,250000000.0
5,john carter,Andrew Stanton,"[Daryl Sabara, Samantha Morton, Polly Walker]",6.6,73058679.0,263700000.0


## Combine datasets

**Pointers about combining datasets:**
- Keep only df2 entries because it has budget values
- Fetch actors from df1 for more actors in common entries

In [10]:
# Merge the two dataframes to keep the common entries
df_merge = df2_relevant.merge(df1_relevant, on = "title")
df_merge.head()

,title,director,actors_x,imdb_rating,revenue,budget,actors_y
0,avatar,James Cameron,"[CCH Pounder, Joel David Moore, Wes Studi]",7.9,760505847.0,237000000.0,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."
1,pirates of the caribbean: at world's end,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Jack Davenport]",7.1,309404152.0,300000000.0,"[Johnny Depp, Orlando Bloom, Keira Knightley, ..."
2,spectre,Sam Mendes,"[Christoph Waltz, Rory Kinnear, Stephanie Sigman]",6.8,200074175.0,245000000.0,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R..."
3,the dark knight rises,Christopher Nolan,"[Tom Hardy, Christian Bale, Joseph Gordon-Levitt]",8.5,448130642.0,250000000.0,"[Christian Bale, Tom Hardy, Anne Hathaway, Gar..."
4,john carter,Andrew Stanton,"[Daryl Sabara, Samantha Morton, Polly Walker]",6.6,73058679.0,263700000.0,"[Taylor Kitsch, Lynn Collins, Willem Dafoe, Sa..."


In [11]:
# Concatenate the actors list from the primary dataset (run this cell only once)
df_merge['actors_x'] = df_merge['actors_x'] + df_merge['actors_y']
df_merge = df_merge.drop(columns = ['actors_y'])

In [12]:
# Standardize column names
df_merge.columns = ['title', 'director', 'actors', 'imdb_rating', 'revenue', 'budget']
# Keep only the distinct actor names
df_merge['actors'] = df_merge['actors'].apply(lambda x: list(set(x)))
df_merge.head()

,title,director,actors,imdb_rating,revenue,budget
0,avatar,James Cameron,"[Sigourney Weaver, Joel David Moore, CCH Pound...",7.9,760505847.0,237000000.0
1,pirates of the caribbean: at world's end,Gore Verbinski,"[Geoffrey Rush, Jack Davenport, Orlando Bloom,...",7.1,309404152.0,300000000.0
2,spectre,Sam Mendes,"[Stephanie Sigman, Christoph Waltz, Léa Seydou...",6.8,200074175.0,245000000.0
3,the dark knight rises,Christopher Nolan,"[Christian Bale, Joseph Gordon-Levitt, Tom Har...",8.5,448130642.0,250000000.0
4,john carter,Andrew Stanton,"[Polly Walker, Willem Dafoe, Lynn Collins, Sam...",6.6,73058679.0,263700000.0


In [13]:
# Get the remaining secondary dataset entries
df2_exclusive = df2_relevant[~df2_relevant['title'].isin(df_merge['title'])]

In [14]:
# Finally, combine the two dataframes
df_comb = df_merge.append(df2_exclusive)
df_comb.head()

,title,director,actors,imdb_rating,revenue,budget
0,avatar,James Cameron,"[Sigourney Weaver, Joel David Moore, CCH Pound...",7.9,760505847.0,237000000.0
1,pirates of the caribbean: at world's end,Gore Verbinski,"[Geoffrey Rush, Jack Davenport, Orlando Bloom,...",7.1,309404152.0,300000000.0
2,spectre,Sam Mendes,"[Stephanie Sigman, Christoph Waltz, Léa Seydou...",6.8,200074175.0,245000000.0
3,the dark knight rises,Christopher Nolan,"[Christian Bale, Joseph Gordon-Levitt, Tom Har...",8.5,448130642.0,250000000.0
4,john carter,Andrew Stanton,"[Polly Walker, Willem Dafoe, Lynn Collins, Sam...",6.6,73058679.0,263700000.0


In [15]:
# df_comb.to_csv(r'tmp1.csv', index = False)

## Create director-actor pairs

In [16]:
# Seperate the actors list into multiple rows
df_comb = df_comb.explode('actors')
# Remove duplicate entries
df_comb = df_comb.drop_duplicates()
df_comb.head()

,title,director,actors,imdb_rating,revenue,budget
0,avatar,James Cameron,Sigourney Weaver,7.9,760505847.0,237000000.0
0,avatar,James Cameron,Joel David Moore,7.9,760505847.0,237000000.0
0,avatar,James Cameron,CCH Pounder,7.9,760505847.0,237000000.0
0,avatar,James Cameron,Michelle Rodriguez,7.9,760505847.0,237000000.0
0,avatar,James Cameron,Zoe Saldana,7.9,760505847.0,237000000.0


In [17]:
# Group by director-actor pair, keep the remaining columns as lists
df_group = df_comb.groupby(['director', 'actors'], as_index=False)[['title', 'imdb_rating', 'revenue', 'budget']].agg(lambda x: list(x))
df_group.head()

,director,actors,title,imdb_rating,revenue,budget
0,Aaron Schneider,Bill Cobbs,[get low],[7.1],[9176553.0],[7500000.0]
1,Aaron Schneider,Bill Murray,[get low],[7.1],[9176553.0],[7500000.0]
2,Aaron Schneider,Robert Duvall,[get low],[7.1],[9176553.0],[7500000.0]
3,Aaron Seltzer,Alyson Hannigan,[date movie],[2.7],[48546578.0],[20000000.0]
4,Aaron Seltzer,Carmen Electra,[date movie],[2.7],[48546578.0],[20000000.0]


In [18]:
# Get the movies count for each pair
df_group['movies_count'] = df_group['title'].apply(len)
# Keep only the pairs with at least 2 movies
df_filter = df_group[df_group.movies_count > 1]
df_filter.head()

,director,actors,title,imdb_rating,revenue,budget,movies_count
26,Adam McKay,John C. Reilly,"[talladega nights: the ballad of ricky bobby, ...","[6.6, 6.9]","[148213377.0, 100468793.0]","[73000000.0, 65000000.0]",2
33,Adam McKay,Steve Carell,"[the big short, anchorman 2: the legend contin...","[7.8, 6.3, 7.2]","[70235322.0, 2175312.0, 84136909.0]","[28000000.0, 50000000.0, 26000000.0]",3
34,Adam McKay,Will Ferrell,"[the other guys, talladega nights: the ballad ...","[6.7, 6.6, 6.9, 6.3, 7.2]","[119219978.0, 148213377.0, 100468793.0, 217531...","[100000000.0, 73000000.0, 65000000.0, 50000000...",5
191,Alejandro G. Iñárritu,Naomi Watts,[birdman or (the unexpected virtue of ignoranc...,"[7.8, 7.7]","[42335698.0, 16248701.0]","[18000000.0, 20000000.0]",2
217,Alex Kendrick,Alex Kendrick,"[courageous, facing the giants]","[7.0, 6.7]","[34522221.0, 10174663.0]","[2000000.0, 100000.0]",2


In [19]:
df_filter.shape

(501, 7)

In [20]:
# df_filter.to_csv(r'tmp2.csv', index = False)

In [21]:
### Store the result into a json file
directors = df_filter['director'].drop_duplicates()
actors = df_filter['actors'].drop_duplicates()

# Iterate rows:
nodes = list()
names = list()
for director in directors:
    nodes.append({'id': director, 'group' : 1})
    names.append(director)
for actor in actors:
    nodes.append({'id': actor, 'group' : 2})
    names.append(actor)

links = list()
for index, row in df_filter.iterrows():
    links.append({"source": row['director'], "target": row['actors'], "title": row['title'], "imdb_rating": row['imdb_rating'],
                  "revenue": row['revenue'], "budget": row['budget'], "counts": row['movies_count']})

json_file = {
    'nodes': nodes,
    'links': links
}

with open('new-result.json', 'w') as fp:
    json.dump(json_file, fp)

## Troublesome cases

#### Common people as director and actor

In [22]:
acting_directors = list(set(directors[directors.isin(actors)]))
len(acting_directors)

19

In [23]:
directing_actors = list(set(actors[actors.isin(directors)]))
len(directing_actors)

19

In [24]:
print(acting_directors)

['Alex Kendrick', 'Ben Stiller', 'Woody Allen', 'Michael Moore', 'Trey Parker', 'Denzel Washington', 'Seth MacFarlane', 'Clint Eastwood', 'Ben Affleck', 'Ben Falcone', 'Jay Chandrasekhar', 'Jonathan Frakes', 'Tom Hanks', 'Warren Beatty', 'Jon Favreau', 'Quentin Tarantino', 'Sylvester Stallone', 'Vincent Gallo', 'Leonard Nimoy']


#### Self-paired people

In [25]:
df_pair = df_filter[['director', 'actors']]
df_pair['same'] = np.where((df_pair.director == df_pair.actors), True, False)

In [26]:
df_pair = df_pair[df_pair.same == True]
self_paired = df_pair['director']
self_paired.shape

(18,)

In [27]:
print(self_paired.values)

['Alex Kendrick' 'Ben Affleck' 'Ben Falcone' 'Ben Stiller'
 'Clint Eastwood' 'Denzel Washington' 'Jay Chandrasekhar' 'Jon Favreau'
 'Jonathan Frakes' 'Leonard Nimoy' 'Michael Moore' 'Seth MacFarlane'
 'Sylvester Stallone' 'Tom Hanks' 'Trey Parker' 'Vincent Gallo'
 'Warren Beatty' 'Woody Allen']
